In [0]:
import json
import random
import pprint
import math

file = open("ulaz1.txt", "r")
file_content = file.read()

tmp = json.loads(file_content)
ucionice_dict = tmp["Ucionice"]
casovi = tmp["Casovi"]

dani = ["PON", "UTO", "SRE", "CET", "PET"]
predmeti_set = set() # {"Bezbednost mreza", "Data centar infrastruktura"...}
nastavnici_set = set() # {"babic Djordje", "Bojovic Petar"...}
grupe_set = set()
nastavnici = []
predmeti = []
grupe = []
ucionice = [] # Niz ucionica ["Atelje", "Kolarac", "U1"..."Studio"]

for u in ucionice_dict:
    tmp = ucionice_dict[u]
    for x in tmp:
      ucionice.append(x)

random.shuffle(ucionice)
# Dodajemo id svakom casu, 0, 1, 2...len(casovi)
# Dodajemo zapravo ucionice "Ucionica" = ["U1", "U2"]
id = 0
for cas in casovi:
  cas["Ucionica"] = ucionice_dict[cas["Ucionica"]]
  predmeti_set.add(cas["Predmet"])
  nastavnici_set.add(cas["Nastavnik"])
  for g in cas["Grupe"]:
    grupe_set.add(g)
  cas["ID"] = id
  id += 1

# Prebacim nastavnike i predmete u listu  
for nastavnik in nastavnici_set:
  nastavnici.append(nastavnik)

for predmet in predmeti_set:
  predmeti.append(predmet)

for grupa in grupe_set:
  grupe.append(grupa)  

zauzeti_nastavnici = [(-1, -1) for _ in range(len(nastavnici))]
zauzete_grupe = [(-1,-1) for _ in range(len(grupe))]
raspored =[[-1 for _ in range(len(ucionice))] for i in range(60)]
generisani_raspored = []
tmp_casovi = [-1 for _ in casovi]

def rep_load():
  tmp = json.loads(file_content)
  ucionice_dict = tmp["Ucionice"]
  casovi = tmp["Casovi"]
  id = 0
  for cas in casovi:
    cas["Ucionica"] = ucionice_dict[cas["Ucionica"]]
    predmeti_set.add(cas["Predmet"])
    nastavnici_set.add(cas["Nastavnik"])
    for g in cas["Grupe"]:
      grupe_set.add(g)
    cas["ID"] = id
    id += 1
    
  return casovi

def nadji_moguce_termine(ucionica, sat, casovi):
  moguci_termini = []
  konacni_termini = []
  for cas in casovi:
    for u in cas["Ucionica"]:
      if(ucionica == u):
        if(sat//12 == (sat + int(cas["Trajanje"]))//12):
          if(zauzeti_nastavnici[nastavnici.index(cas["Nastavnik"])][1] <= sat or
             sat + int(cas["Trajanje"]) <= zauzeti_nastavnici[nastavnici.index(cas["Nastavnik"])][0]):
              moguci_termini.append(cas)

  flag = True
  for cas in moguci_termini:
    for g in cas["Grupe"]:
      if(zauzete_grupe[grupe.index(g)][1] > sat or
          (sat < zauzete_grupe[grupe.index(g)][0] and sat + int(cas["Trajanje"]) > zauzete_grupe[grupe.index(g)][0])):
        flag = False
    if flag == True:
      konacni_termini.append(cas)
      
  """  
  sortirani = sorted(konacni_termini, key=lambda k: k["Trajanje"])    
  apsolutno_konacni = [t for t in sortirani if int(t["Trajanje"]) > 2]
  """
  sortirani = sorted(konacni_termini, key=lambda k: len(k["Grupe"]))   
  apsolutno_konacni = [t for t in sortirani if len(t["Grupe"]) > 4]
  
  if len(apsolutno_konacni) != 0:
    return apsolutno_konacni
  return konacni_termini


populacija = []
hodovi = []
population_size = 20
moj_json = []

while len(populacija) < 20:
  iteracija = 0
  casovi = rep_load()
  random.shuffle(casovi)
  zauzeti_nastavnici = [(-1, -1) for _ in range(len(nastavnici))]
  zauzete_grupe = [(-1, -1) for _ in range(len(grupe))]
  raspored =[[-1 for _ in range(len(ucionice))] for i in range(60)]
  x = [[-1 for i in range(1)] for j in range(len(nastavnici))] 
  y = [[-1 for i in range(1)] for j in range(len(grupe))] 
  generisani_raspored = []
  tmp_casovi = [-1 for _ in casovi]
  flag = True
  
  while len(casovi) != 0:
    iteracija += 1    
    if (iteracija > 50):
      flag = False
      break
    for i in range(60):
      for j in range(len(ucionice)):
        if(raspored[i][j] == -1): # Prazna ucionica
          moguci_casovi = nadji_moguce_termine(ucionice[j],i, casovi)
          if len(moguci_casovi) != 0:
            index = random.randrange(0, len(moguci_casovi))
            izabrani_termin = moguci_casovi[index]

            trajanje = int(izabrani_termin["Trajanje"])

            zauzeti_nastavnici[nastavnici.index(izabrani_termin["Nastavnik"])] = (i, i + trajanje) 
            x[nastavnici.index(izabrani_termin["Nastavnik"])].append((i, i + trajanje))

            for g in izabrani_termin["Grupe"]:
              zauzete_grupe[grupe.index(g)] = (i, i + trajanje)
              y[grupe.index(g)].append((i, i + trajanje))

            for t in range(trajanje):
              raspored[i + t][j] = int(izabrani_termin["ID"])

            izabrani_termin["Zadata ucionica"] = str(ucionice[j])
            izabrani_termin["Vreme pocetka"] = str(i + 9 - 13*(i//12))
            izabrani_termin["Dan"] = str(dani[i//12])

            generisani_raspored.append(izabrani_termin)
            casovi.remove(izabrani_termin)
            tmp_casovi[int(izabrani_termin["ID"])] = izabrani_termin

  if flag:  
    populacija.append(raspored)
    hodovi.append((x, y))
    moj_json.append(generisani_raspored)
      
def izracunaj_prazan_hod(hodovi):
  iteracija = -1
  prosek_grupa = 0
  prosek_nastavnika = 0
  for(x, y) in hodovi:
    prazan_hod_nastavnika = 0
    prazan_hod_grupe = 0
    for i in range(len(nastavnici)):
        for j in range(1, len(x[i]) - 1):
          if x[i][j] != -1:
            if (x[i][j][1] < x[i][j+1][0]) and (x[i][j][1]//12 == x[i][j+1][0]//12):
              prazan_hod_nastavnika += x[i][j+1][0] - x[i][j][1]
              
    for i in range(len(grupe)):
      for j in range(1, len(y[i]) - 1):
        if y[i][j] != -1:
          if (y[i][j][1] < y[i][j+1][0]) and (y[i][j][1]//12 == y[i][j+1][0]//12):
            prazan_hod_grupe += y[i][j+1][0] - y[i][j][1]
          
    iteracija += 1
    print("Gubitak za ", iteracija, ". iteraciju")  
    print("Prazan hod nastavnika: {:.3f}".format(prazan_hod_nastavnika/len(nastavnici)))
    print("Prazan hod grupe {:.3f}".format(prazan_hod_grupe/len(grupe)))
    print("-------------------------")
    prosek_grupa += prazan_hod_grupe/len(grupe)
    prosek_nastavnika += prazan_hod_nastavnika/len(nastavnici)
  
  print("Prosek za celu populaciju \n Prosek grupe: {}, \n Prosek nastavnika: {}".format(prosek_grupa/len(grupe), prosek_nastavnika/len(nastavnici)))  
  
  
print("Uspeo sam da napravim ", len(populacija), " rasporeda")
izracunaj_prazan_hod(hodovi)

In [0]:
with open('raspored.txt', 'w') as outfile:  
    json.dump(moj_json, outfile)

In [0]:

def proveri_preklapanja(raspored, tmp_casovi):
  checked = []
  preklapanja = 0
  for i in range(60):
    for j in range(len(ucionice)):
      
      index = raspored[i][j]
      
      if(index != -1 and index not in checked): 
        
        checked.append(index)
        tmp_grupe = set(tmp_casovi[index]["Grupe"])
        
        submatrix = raspored[i:min(i + int(tmp_casovi[index]["Trajanje"]), 59)][:]
        sub_checked = []

        for m in range(len(submatrix)):
          for n in range(len(submatrix[0])):
            
            if(submatrix[m][n] != -1):
              poredi_sa = submatrix[m][n]
              if poredi_sa not in sub_checked and poredi_sa not in checked:
                sub_checked.append(poredi_sa)
                poredi_grupe = set(tmp_casovi[poredi_sa]["Grupe"])
                presek = tmp_grupe.intersection(poredi_grupe)
                preklapanja += len(presek)
                
                if len(presek) != 0:
                  print("----Grupa----")
                  print(tmp_casovi[index])
                  print(tmp_casovi[poredi_sa])
                  print(tmp_casovi[index]["ID"], tmp_casovi[poredi_sa]["ID"])
                  pprint.pprint(submatrix)
                  for t in presek:
                    print(zauzete_grupe[grupe.index(t)])
                if tmp_casovi[index]["Nastavnik"] == tmp_casovi[poredi_sa]["Nastavnik"]:
                  preklapanja += 1
                  """
                  print("-----Profesor----")
                  print(tmp_casovi[index])
                  print(tmp_casovi[poredi_sa])
                  print(tmp_casovi[index]["ID"], tmp_casovi[poredi_sa]["ID"])
                  pprint.pprint(submatrix)
                  """
                
  return preklapanja